# VacationPy
---

In [12]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [13]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv('../output_data/cities.csv')

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,bilibino,68.0546,166.4372,18.15,58,3,0.78,RU,clear sky
1,1,hermanus,-34.4187,19.2345,14.89,82,97,4.99,ZA,overcast clouds
2,2,badger,64.8000,-147.5333,17.12,84,100,2.06,US,overcast clouds
3,3,margaret river,-33.9500,115.0667,16.23,69,100,6.74,AU,overcast clouds
4,4,kodiak,57.7900,-152.4072,11.73,94,75,0.00,US,broken clouds


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [14]:
%%capture --no-display

# Configure the map plot
city_map = city_data_df.hvplot.points("Lat",
                                      "Lng",
                                     geo = True,
                                     color ="City",
                                     alpha =0.6,
                                     size= "Humidity",
                                     tiles = "OSM")

# Display the map
city_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lat,Lng]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [15]:
# Narrow down cities that fit criteria and drop any results with null values

#criteria to filter
#max temperature lower than 27 degrees but higher than 21
#wind speed less than 4.5 m/s
#zero cloudiness

weather_filtered_df = city_data_df[(city_data_df["Max Temp"] <27)
            & (city_data_df["Max Temp"] > 21)
            & (city_data_df["Wind Speed"] < 4.5)
            & (city_data_df["Cloudiness"] < 1)]
                        

# Drop any rows with null values
weather_filtered_df.dropna()

# Display sample data
weather_filtered_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
30,30,hawaiian paradise park,19.5933,-154.9731,21.13,87,0,3.09,US,clear sky
91,91,walvis bay,-22.9575,14.5053,26.53,34,0,1.54,NaN,clear sky
147,147,haiku-pauwela,20.9219,-156.3051,23.91,76,0,2.57,US,clear sky
171,171,craig,40.5153,-107.5464,22.97,28,0,2.06,US,clear sky
231,231,balqash,46.8481,74.9950,24.79,42,0,2.76,KZ,clear sky


### Step 3: Create a new DataFrame called `hotel_df`.

In [16]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = weather_filtered_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
30,hawaiian paradise park,US,19.5933,-154.9731,87,
91,walvis bay,NaN,-22.9575,14.5053,34,
147,haiku-pauwela,US,20.9219,-156.3051,76,
171,craig,US,40.5153,-107.5464,28,
231,balqash,KZ,46.8481,74.9950,42,
235,huron,US,41.1501,-82.5499,57,
315,kerch,UA,45.3583,36.4758,51,
355,qingdao,CN,36.0986,120.3719,78,
371,shitanjing,CN,39.2342,106.3439,18,
387,alamosa,US,37.4695,-105.8700,37,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [17]:
# Set parameters to search for a hotel
radius = 10000
params = {"categories": "accomodation.hotel",
          "apiKey": geoapify_key}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    longitude = row["Lng"]
    latitude = row["Lat"]
        
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params = params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
hawaiian paradise park - nearest hotel: No hotel found
walvis bay - nearest hotel: No hotel found
haiku-pauwela - nearest hotel: No hotel found
craig - nearest hotel: No hotel found
balqash - nearest hotel: No hotel found
huron - nearest hotel: No hotel found
kerch - nearest hotel: No hotel found
qingdao - nearest hotel: No hotel found
shitanjing - nearest hotel: No hotel found
alamosa - nearest hotel: No hotel found
tongchuanshi - nearest hotel: No hotel found
stjordalshalsen - nearest hotel: No hotel found
shijiazhuang - nearest hotel: No hotel found
ankang - nearest hotel: No hotel found
saint-francois - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
30,hawaiian paradise park,US,19.5933,-154.9731,87,No hotel found
91,walvis bay,NaN,-22.9575,14.5053,34,No hotel found
147,haiku-pauwela,US,20.9219,-156.3051,76,No hotel found
171,craig,US,40.5153,-107.5464,28,No hotel found
231,balqash,KZ,46.8481,74.9950,42,No hotel found
235,huron,US,41.1501,-82.5499,57,No hotel found
315,kerch,UA,45.3583,36.4758,51,No hotel found
355,qingdao,CN,36.0986,120.3719,78,No hotel found
371,shitanjing,CN,39.2342,106.3439,18,No hotel found
387,alamosa,US,37.4695,-105.8700,37,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [18]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points("Lng",
                                  "Lat",
                                  geo = True,
                                  color = "City",
                                  alpha = 0.8,
                                  size = "Humidity",
                                  tiles = "OSM",
                                  hover_cols = ["Hotel Name", "Country"]
                                  )

# Display the map
hotel_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)